In [2]:
import os
import torch
from accelerate import init_empty_weights, Accelerator
from transformers import AutoModelForCausalLM, AutoTokenizer

In [3]:
accelerator = Accelerator()

In [4]:
base_path = '/home/hrant/chem/tigran/ChemLactica/checkpoints/facebook'
model_size = 'galactica-125m'
run_hash = 'ac7915df73b24ee3a4e172d6'
checkpoint_no = 'checkpoint-253952'
checkpoint_path = os.path.join(base_path, model_size, run_hash, checkpoint_no)
checkpoint_path

'/home/hrant/chem/tigran/ChemLactica/checkpoints/facebook/galactica-125m/ac7915df73b24ee3a4e172d6/checkpoint-253952'

In [5]:
# change this to a free GPU device for faster inference
device = "cuda:5"
# device = "cpu"

In [6]:
# run_hash = 'dff5ace8f2ac45cc9681ab6a'
# checkpoint_no = 'checkpoint-194560'
# checkpoint_path = os.path.join(base_path, model_size, run_hash, checkpoint_no)

In [7]:
tokenizer = AutoTokenizer.from_pretrained("./tokenizer/ChemLacticaTokenizer/")
print('tokenizer size: ', len(tokenizer))
# model = AutoModelForCausalLM.from_pretrained(checkpoint_path)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


tokenizer size:  50029


In [27]:
model = AutoModelForCausalLM.from_pretrained(checkpoint_path, torch_dtype=torch.bfloat16)
model.to(device)
assert(model.model.decoder.embed_tokens.num_embeddings + 1 == len(tokenizer))

In [28]:
prompt = """</s>[QED 0.84][START_SMILES]"""
prompt = """</s>[SAS 4.32][START_SMILES]"""
prompt = """</s>[TPSA 4.32][START_SMILES]"""

In [29]:
inputs = tokenizer(prompt, return_tensors="pt")
inputs = inputs.to(device)
inputs = inputs.to(torch.bfloat16)

Attempting to cast a BatchEncoding to type torch.bfloat16. This is not supported.


In [30]:
outputs = model.generate(inputs.input_ids, max_new_tokens=300, do_sample=True, top_k=1, top_p=0.95)
tokenizer.batch_decode(outputs, skip_special_tokens=False)

['</s>[TPSA 4.32][START_SMILES]CC1=CC(=C(C=C1)C)N2C(=O)C3=C(C2=O)C=C(C=C3)Br[END_SMILES][CID 17131262][NUMROTATABLEBONDS 1][SAS 1.86][IUPAC 5-bromo-2-(2,5-dimethylphenyl)isoindole-1,3-dione][NUMAROMATICRINGS 2][WEIGHT 330.0][NHOHCOUNT 0][NUMSATURATEDRINGS 0][NUMALIPHATICHETEROCYCLES 1][RINGCOUNT 3][NUMAROMATICCARBOCYCLES 2][NUMALIPHATICRINGS 1][NUMSATURATEDHETEROCYCLES 0][NUMHDONORS 0][NUMALIPHATICCARBOCYCLES 0][HEAVYATOMCOUNT 20][NUMHETEROATOMS 4][NUMHACCEPTORS 2][FRACTIONCSP3 0.13][NOCOUNT 3][QED 0.75][CLOGP 3.8][NUMAROMATICHETEROCYCLES 0][NUMSATURATEDCARBOCYCLES 0]</s>']

In [100]:
import rdkit.Chem as Chem
import sys
from rdkit.Chem import RDConfig, MACCSkeys
import numpy as np 
def tanimoto_similarity(smiles_1, smiles_2):
    fgp1 = np.array(MACCSkeys.GenMACCSKeys(Chem.MolFromSmiles(smiles_1)))
    fgp2 = np.array(MACCSkeys.GenMACCSKeys(Chem.MolFromSmiles(smiles_2)))

    both = np.sum(fgp1 & fgp2)

    return both / (np.sum(fgp1) + np.sum(fgp2) - both)
sys.path.append(os.path.join(RDConfig.RDContribDir, 'SA_Score'))
import sascorer
mol_source = Chem.MolFromSmiles("CC1=C(C(=C(C(=C1C)C)C2=C(C(=C(C(=C2C)C)C3=C(C(=C(C(=C3C)C)C)C)C)C)C)C)C")
mol_second = Chem.MolFromSmiles("CC1=CC(=C(C=C1)C)C(=C)C2=CC=CC=C2C")
source_sas = sascorer.calculateScore(mol_source)
print(tanimoto_similarity("CC1=C(C(=C(C(=C1C)C)C2=C(C(=C(C(=C2C)C)C3=C(C(=C(C(=C3C)C)C)C)C)C)C)C)C","CC1=CC(=C(C=C1)C)C(=C)C2=CC=CC=C2C"))

0.6666666666666666
